In [1]:
%run ./preprocessing.ipynb

You are using pip version 8.1.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dataset = dataset()

In [ ]:
booksdf = dataset.toDF(['author', 'description', 'edition', 'format', 
                    'isbn13', 'pages', 'rating', 'ratingCount', 
                    'review_count', 'title', 'genres', 'image_url']) \
    .drop("edition") \
    .drop("format") \
    .drop("pages") \
    .drop("isbn13") \
    .drop("review_count") \
    .drop("image_url")

booksdf = booksdf.withColumn("genre", booksdf["genres"]) 
booksdf = booksdf.dropna(subset=("genres", 'genre'))
booksdf.printSchema()

hashingTF = HashingTF(inputCol="description", outputCol="rawFeatures", numFeatures=32)
featurizedData = hashingTF.transform(booksdf)

featurizedData.select('description', 'rawFeatures').show(10)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("title", "genres", "features").show(10)

(trainingData, testData) = rescaledData.randomSplit([0.7, 0.3])

evaluator = MulticlassClassificationEvaluator(labelCol="genre", predictionCol="prediction", metricName="accuracy")

dt = DecisionTreeClassifier(labelCol="genre", featuresCol="features", cacheNodeIds = True, checkpointInterval = 10, impurity = 'entropy')
dtModel = dt.fit(trainingData)


dtPredictions = dtModel.transform(testData)
dtPredictions.select("prediction", "genres", "features").show(10)

dtAccuracy = evaluator.evaluate(dtPredictions)
print(dtAccuracy)


root
 |-- author: string (nullable = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rating: string (nullable = true)
 |-- ratingCount: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: double (nullable = true)
 |-- genre: double (nullable = true)

+--------------------+--------------------+
|         description|         rawFeatures|
+--------------------+--------------------+
|[alaska, unforgiv...|(32,[0,1,2,3,4,5,...|
|[house, helios, g...|(32,[0,1,2,3,4,5,...|
|[anna, fox, lives...|(32,[1,2,3,5,6,7,...|
|[killed, motherth...|(32,[0,1,2,4,5,6,...|
|[newlyweds, celes...|(32,[1,2,4,5,6,7,...|
|[read, book, make...|(32,[0,1,2,3,4,5,...|
|[knew, date, deat...|(32,[1,2,3,4,5,6,...|
|[unspeakable, cri...|(32,[0,1,2,3,5,6,...|
|[heartwarming, re...|(32,[0,1,2,3,4,5,...|
|[years, rumors, h...|(32,[1,2,3,4,5,8,...|
+--------------------+--------------------+
only showing top 10 rows

+--------------------+------+--